In [9]:
import numpy as np

In [82]:
def getTs(t,lamda_max):
    while True:
        t = t - (np.log(np.random.uniform())/lamda_max)
        if np.random.uniform() <= np.random.poisson(t)/lamda_max:
            return t

In [83]:
def simular(duracion_simulacion,lamda_max,velocidad,no_cpus):
    t = 0
    Na = 0
    Nd = 0
    n = 0
    A = {}
    D = {}
    ta = getTs(t,lamda_max)
    td = np.infty
    while True:
        if min(ta,td) <= duracion_simulacion:
            if ta <= td:
                t = ta
                Na += 1
                n += 1
                ta = getTs(t,lamda_max)
                if n==1:
                    td = t + np.random.exponential(velocidad)
                A[Na] = t
            else:
                t = td
                n -= 1
                Nd += 1
                if n==0:
                    td = np.infty
                else:
                    td = t + np.random.exponential(velocidad)
                D[Nd] = t
        else:
            if n > 0:
                t = td
                n -= 1
                Nd += 1
                if n > 0:
                    td = t + np.random.exponential(velocidad)
                D[Nd] = t
            else:
                Tp = max(t-duracion_simulacion,0)
                break;
    return {"Na": Na, "Nd": Nd, "n":n, "Tp": Tp, "A": A, "D": D}

In [84]:
max_solicitudes = [2400,6000]
duraciones = [60]
Proveedores = [
    {
        "nombre": "Gorilla Megacomputing",
        "no_cpus": 1,
        "velocidad": 100
    },
    {
        "nombre": "Ants smart computing",
        "no_cpus": 10,
        "velocidad": 10
    },
]

In [85]:
for duracion_simulacion in duraciones:
    for lamda_max in max_solicitudes:
        for proveedor in Proveedores:
            resultados = simular(duracion_simulacion,lamda_max,proveedor["velocidad"],proveedor["no_cpus"])

In [ ]:
Na = [] # número de llegadas al tiempo t
Nd = [] # número de salidas al tiempo t
n = [] # número de solicitudes en el sistema en el tiempo t
for i in range(len(Proveedores)):
    Na.append(np.empty((0,2))) 
    Nd.append(np.empty((0,2))) 
    n.append(np.empty((0,2)))